В домашней работе вам необходимо создать интерфейс к вашей модели нейронной сети, используя Streamlit (можно и Gradio). Для этого:

1. Определитесь, какую задачу будет решать ваша нейронная сеть.
2. Продумайте интерфейс взаимодействия с пользователем, какими параметрами модели пользователь будет управлять.
3. Обучите модель на любом публичном датасете или возьмите из любого предыдущего урока. Вспомните как происходит загрузка и выгрузка моделей в Keras.
4. Загрузите обученную модель в Colab с интерфейсом (деплой модели).
5. Создайте интерфейс для инференса вашей модели (для запросов к модели).
6. Изучите как происходит загрузка файлов для моделей с помощью Streamlit по [ссылке](https://docs.streamlit.io/develop/api-reference/widgets/st.file_uploader).
7. Добавьте в интерфейс возможность загрузки пользовательских данных для инференса. Это может быть текстовый файл, картинка, аудиофайл или др.
8. Выполнив задание, получите 3 балла.
9. Вы также можете получить дополнительные 2 балла, если реализуете в одном интерфейсе обучение модели и её инференс.

**Инференс** - это процесс исполнения обученных моделей машинного обучения для получения предсказаний на данных, поданных на вход модели.
Обычно нейронная сеть проходит три жизненных этапа: обучение, деплой и инференс. Инференсом называется непрерывная работа какой-либо нейронной сети на конечном устройстве.

**Деплой** - загрузка на сервер.

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 942.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.6 MB/s eta 0:00:00


In [ ]:
%%writefile main.py
import os
import shutil
from keras import layers, Input, Model
from keras import models
import keras
import numpy as np
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNet
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
import matplotlib.pyplot as plt

import requests
import zipfile
import streamlit as st

IMAGE_PATH = './temp/PetImages/'
BASE_DIR = './dataset/'

# Создание путей для тренировки, валид. и тестов
train_dir = os.path.join(BASE_DIR, 'train')
validation_dir = os.path.join(BASE_DIR, 'validation')
test_dir = os.path.join(BASE_DIR, 'test')

if not os.path.exists(BASE_DIR):
    os.mkdir(BASE_DIR)
    os.mkdir(train_dir)
    os.mkdir(validation_dir)
    os.mkdir(test_dir)

IMG_WIDTH = 224
IMG_HEIGHT = 224

st.set_page_config(layout="wide")
leftl_column, left_column, right_column = st.columns((1, 2, 2))
leftl_column.subheader('Информационное окно')
left_column.subheader('Обучение модели')
right_column.subheader('Работа с обученной моделью')


# Скачать и разархивировать датасет
def get_dataset():
    leftl_column.write('Идет загрузка файла')
    response = requests.get(st.session_state.url)
    file_Path = 'dataset.zip'

    if response.status_code == 200:
        with open(file_Path, 'wb') as file:
            file.write(response.content)
        leftl_column.write('Файл скачан успешно')

        leftl_column.write('Идет разархивирование файла')
        with zipfile.ZipFile(file_Path, 'r') as zip_ref:
            zip_ref.extractall("temp")
        leftl_column.write('Разархивирование файла закончено')

        CLASS_LIST = sorted(os.listdir(IMAGE_PATH))
        leftl_column.write(f"Датасет содержит классы: {CLASS_LIST}")

    else:
        leftl_column.write('Ошибка загрузки')


def create_dataset(img_path: str, new_path: str, class_name: str, start_index: int, end_index: int):
    src_path = os.path.join(img_path, class_name)  # Полный путь к папке с изображениями класса
    dst_path = os.path.join(new_path, class_name)  # Полный путь к папке с новым датасетом класса

    # Получение списка имен файлов с изображениями текущего класса
    class_files = os.listdir(src_path)
    # Создаем подпапку, используя путь
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)

    for fname in class_files[start_index: end_index]:
        src = os.path.join(src_path, fname)
        dst = os.path.join(dst_path, fname)
        shutil.copyfile(src, dst)


# Ввод URL
left_column.text_input("Введите URL адрес Датасета (записан по умолчанию)",
                       "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip",
                       key="url")

# Кнопка загрузки датасет
if left_column.button('Загрузить датасет'):
    get_dataset()

# Кнопка содания выборок
left_column.divider()
btn_create = left_column.button('Создать выборки')
# бегунок выбора процентов для выборки
percent = left_column.slider('Выберите процент для тренировочной выборки', value=70)

if btn_create:
    if os.path.exists("temp"):
        leftl_column.write("Запуск создания выборок")

        CLASS_LIST = sorted(os.listdir(IMAGE_PATH))
        NUM_CLASSES = len(CLASS_LIST)

        num_skipped = 0 # счетчик поврежденных файлов
        for folder_name in os.listdir(IMAGE_PATH): # перебираем папки
            folder_path = os.path.join(IMAGE_PATH, folder_name) # склеиваем путь
            for fname in os.listdir(folder_path): # получаем список файлов в папке
                fpath = os.path.join(folder_path, fname) # получаем путь до файла
                try:
                    fobj = open(fpath, "rb") # пытаемся открыть файл для бинарного чтения (rb)
                    is_jfif = b"JFIF" in fobj.peek(10)
                finally:
                    fobj.close()

                if not is_jfif:
                    num_skipped += 1
                    os.remove(fpath)
        leftl_column.write(f"Удалено изображений без признака b'JFIF': {num_skipped}")

        # Расчет процентов (на валидацию и тестовую, пусть распределяется поровну)
        dataset_len = sum(len(subdir[2]) for subdir in os.walk(IMAGE_PATH))
        train_len = int(dataset_len * percent / 100)
        val_test_len = int((dataset_len - train_len) * 50 / 100)

        for class_label in range(NUM_CLASSES):
            class_name = CLASS_LIST[class_label]
            class_path = IMAGE_PATH + class_name
            class_len = os.listdir(class_path)
            leftl_column.write(f'Размер класса {class_name} составляет {len(class_len)} животных')

            # Создание выборок
            create_dataset(IMAGE_PATH, train_dir, class_name, 0, train_len // NUM_CLASSES)
            create_dataset(IMAGE_PATH, validation_dir, class_name, train_len // NUM_CLASSES, (train_len + val_test_len) // NUM_CLASSES)
            create_dataset(IMAGE_PATH, test_dir, class_name, (train_len + val_test_len) // NUM_CLASSES, None)

        leftl_column.write(f'Общий размер базы для обучения: {dataset_len}')

        leftl_column.write(f"Число кошек {len(os.listdir(os.path.join(train_dir, 'Cat')))}, "
         f"число собак {len(os.listdir(os.path.join(train_dir, 'Dog')))} в обучающей выборке")

        leftl_column.write(f"Число кошек {len(os.listdir(os.path.join(validation_dir, 'Cat')))}, "
         f"число собак {len(os.listdir(os.path.join(validation_dir, 'Dog')))} в проверочной выборке")

        leftl_column.write(f"Число кошек {len(os.listdir(os.path.join(test_dir, 'Cat')))}, "
         f"число собак {len(os.listdir(os.path.join(test_dir, 'Dog')))} в контрольной выборке")

        if not os.path.exists("model_maker"):
          os.mkdir("model_maker")

        leftl_column.write("Выборки созданы")

    else:
        leftl_column.write("Сначала загрузите Датасет")


def model_maker():
    CLASS_LIST = sorted(os.listdir(IMAGE_PATH))
    NUM_CLASSES = len(CLASS_LIST)

    base_model = MobileNet(include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

    for layer in base_model.layers[:]:
        layer.trainable = False

    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)

    return Model(inputs=input, outputs=predictions)

# Кнопка содания модели и запуска обучения + выбор параметров
left_column.divider()
btn_start = left_column.button('Запустить обучение')
epochs = left_column.slider('Выберите количество эпох', 1, 100, value = 1)
batch_size = left_column.slider('Выберите размер батча', 16, 256, step = 16, value = 256)
learning_rate = left_column.radio('Скорость обучения:', [1e-03, 1e-04, 1e-05])

if btn_start:
    if os.path.exists("model_maker"):
        leftl_column.write("Создание модели")
        model = model_maker()
        model.summary()

        train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
        )

        test_datagen = ImageDataGenerator(rescale=1. / 255)
        train_generator = train_datagen.flow_from_directory(train_dir, target_size=(IMG_WIDTH, IMG_HEIGHT), batch_size=batch_size,
                                                            class_mode='categorical')
        validation_generator = test_datagen.flow_from_directory(validation_dir, target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                                batch_size=batch_size, class_mode='categorical')

        leftl_column.write("Компиляция модели")
        model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=learning_rate), metrics=['acc'])

        leftl_column.write("Обучение началось")
        history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator)
        leftl_column.write("Обучение закончилось")


        def show_history(store):
            acc = store.history['acc']
            val_acc = store.history['val_acc']
            loss = store.history['loss']
            val_loss = store.history['val_loss']
            epochs = range(1, len(acc) + 1)

            fig, axs = plt.subplots(2, 1, figsize=(9, 15), sharey=True)

            axs[0].set_title("График точности на проверочной и обучающей выборках")
            axs[0].plot(epochs, acc, 'r', label='Точность на обучающей выборке')
            axs[0].plot(epochs, val_acc, 'bo', label='Точность на проверочной выборке')

            axs[1].set_title("График потерь на проверочной и обучающей выборках")
            axs[1].plot(epochs, loss, 'r', label='Потери на обучающей выборке')
            axs[1].plot(epochs, val_loss, 'bo', label='Потери на проверочной выборке')
            leftl_column.pyplot(fig)


        show_history(history)

        leftl_column.write("Запуск контрольной выборки")
        test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=batch_size,
        class_mode='categorical'
        )

        test_loss, test_acc = model.evaluate(test_generator)
        leftl_column.write(f'Точность на контрольной выборке: {test_acc}')

        model_name = 'my_model.h5'
        model.save(model_name)
        leftl_column.write(f"Файл {model_name} сохранен в Colab")

    else:
        leftl_column.write("Сначала создайте выборки")

# Кнопка сохранить модель на компьютер
# if left_column.button('Сохранить модель'):
    # if os.path.exists("my_model.h5"):
          # files.download('my_model.h5')
    # else:
    #     leftl_column.write("Сначала пройдите обучение")

# Кнопка загрузить модель с компьютера
data_file = right_column.file_uploader("Загрузить файл c названием 'my_model.h5' (у вас так же должен быть подгружен датасет, для понимания количества классов)",
                                       type=["h5"])

if data_file:
    if os.path.exists("temp"):
        file_details = {"filename": data_file.name, "filetype": data_file.type, "filesize": data_file.size}
        leftl_column.write(file_details)

        with open(data_file.name, "wb") as f:
            f.write(data_file.getbuffer())

        leftl_column.write("Модель сохранена в Colab, можно тестировать модель")

    else:
        leftl_column.write("Сначала загрузите датасет!")

# Запуск теста модели
right_column.divider()
image_file = right_column.file_uploader("Загрузить файл c кошкой или собакой", type=["png","jpg","jpeg"])

if image_file:
    if os.path.exists("my_model.h5"):
        CLASS_LIST = sorted(os.listdir(IMAGE_PATH))

        with open(image_file.name, "wb") as f:
            f.write(image_file.getbuffer())

        img = keras.utils.load_img(image_file.name, target_size=(IMG_WIDTH, IMG_HEIGHT)) # Загружаем картинку
        leftl_column.image(img)

        img_array = keras.utils.img_to_array(img) # Преобразуем картинку в тензор
        img_array = keras.backend.expand_dims(img_array, 0)  # Создание дополнительного измерения для батча

        # Загрузить сохраненную модель
        model = model_maker()
        model = keras.saving.load_model('my_model.h5')
        # model.summary()

        predictions = model.predict(img_array)
        leftl_column.write(predictions)

        leftl_column.write("Предсказание: %s \n Вероятность: %2.1f%%" %
        (CLASS_LIST[np.argmax(predictions)], np.max(predictions)*100))

    else:
        leftl_column.write("Сначала запустите обучение или загрузите готовую модель!")

Writing main.py


In [ ]:
!streamlit run main.py --server.address=localhost >/content/logs.txt & ssh -o "StrictHostKeyChecking no" -R 80:localhost:8501 serveo.net

ssh: connect to host serveo.net port 22: Connection refused
